<a href="https://colab.research.google.com/github/jugernaut/ProgramacionEnParalelo/blob/desarrollo/MPI/03_Algoritmos_MPI_SCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color="Teal" face="Comic Sans MS,arial">
  <h1 align="center"><i>Algoritmos MPI</i></h1>
  </font>
  <font color="Black" face="Comic Sans MS,arial">
  <h5 align="center"><i>Profesor: M.en.C. Miguel Angel Pérez León</i></h5>
    <h5 align="center"><i>Ayudante: Jesús Iván Coss Calderón</i></h5>
    <h5 align="center"><i>Ayudante: Mario Arturo</i></h5>
  <h5 align="center"><i>Materia: Seminario de programación en paralelo</i></h5>
  </font>


# Introducción

Ya que hemos visto algunos métodos interesantes previamente (Monte Carlo) el siguiente paso consiste en convertir la versión secuencial de estos algoritmos a su versión en paralelo.

Para ello vamos a comenzar con el ejemplo básico de la aproximación de $\pi$ empleando métodos de Monte Carlo, tal como se mostró en la presentación previa, en la cual se describe el desarrollo y marco teórico de dicho método.


# Aproximación de $\pi$ mediante Monte Carlo

Para la implementación de este algoritmo en paralelo utilizaremos *MPI* ya que tanto el método y el algoritmo son ideales para tal propósito.

Vale la pena aclarar que la idea de los métodos de Monte Carlo se basan en la generación de números aleatorios y es por esto que *MPI* es muy buena idea para este tipo de algoritmos, ya que *MPI* originalmente se creo para arquitecturas sin memoria compartida (envío de mensajes).

A continuación podemos ver el algoritmo para la aproximación de $\pi$ haciendo uso de *MPI*.

In [ ]:
codigo = """
#include <stdio.h>
#include <stdlib.h>
#include <mpi.h>
#include <math.h>
#include <time.h>
void Validar_Entrada(int argc, int id);
long Lanzamientos(long numLanzamientoProc, int miRango);
 
int main(int argc, char* argv[]){
   // Bloque de variables
   int lan, id_procesador, num_procesadores, total_dardos_dentro, dardo_dentro_proc, lanzamiento_procesador;
   double tiempo_local, tiempo_final, tiempo_inicial, transcurrido, N, pi_aprox;
   MPI_Init(&argc, &argv);
   MPI_Comm_rank(MPI_COMM_WORLD, &id_procesador);
   MPI_Comm_size(MPI_COMM_WORLD, &num_procesadores);
   // se valida la entrada del usuario
   Validar_Entrada(argc, id_procesador);
   // Se guarda el numero total de lanzamientos
   sscanf(argv[1], "%lf", &N);
   // Bloqueo para esperar que cada procesador realice esta parte
   MPI_Barrier(MPI_COMM_WORLD);
   // tiempo inicial
   tiempo_inicial = MPI_Wtime();
   // lanzamiento por procesador
   lanzamiento_procesador = N/num_procesadores;
   // cada procesador realiza esta seccion
   dardo_dentro_proc = Lanzamientos(lanzamiento_procesador, id_procesador);
   // tiempo final
   tiempo_final = MPI_Wtime();
   // tiempo que tomo en cada procesador
   tiempo_local = tiempo_final-tiempo_inicial;
   // suma de cada dardo dentro de la circunferencia
   MPI_Allreduce(&dardo_dentro_proc, &total_dardos_dentro, 1, MPI_INT, MPI_SUM, MPI_COMM_WORLD);
   // nos quedamos con el maximo de los tiempos, ya que eso fue lo que tomo
   MPI_Allreduce(&tiempo_local, &transcurrido, 1, MPI_DOUBLE, MPI_MAX, MPI_COMM_WORLD); 
   // el proceso maestro se encarga de imprimir el resultado
   if (id_procesador == 0) { 
      pi_aprox = (total_dardos_dentro*4)/((double)N);
      lan = (int)N;
      printf("Numero de lanzamientos:      	    %d\\n", lan);
      printf("Valor aproximado de pi:       %24.16f\\n", pi_aprox);
      printf("Maximo tiempo transcurrido del procesador en segundos:  %.16f\\n", transcurrido*100);
   }
   // se liberan los recursos utilizados
   MPI_Finalize();
   return 0;
}

// Funciona que valida la entrada del usuario
void Validar_Entrada(int argc, int id){
   // Validacion de la entrada
   if( argc !=2){
      if (id==0){
         fprintf(stderr,"Incorrect NARGIN\\n");
         fflush(stderr);
      }
   MPI_Abort(MPI_COMM_WORLD,1);
   }  
}

// Funcion que simula los lanzamientos de los dardos
long Lanzamientos(long numLanzamientoProc, int id_proc){
   long lanzamiento, numeroEnCirculo = 0;        
   double x,y;
   unsigned int semilla = (unsigned) time(NULL);
   srand(semilla + id_proc);
   for (lanzamiento = 0; lanzamiento < numLanzamientoProc; lanzamiento++) {
	x = rand_r(&semilla)/(double)RAND_MAX;
	y = rand_r(&semilla)/(double)RAND_MAX;
	if((x*x+y*y) <= 1.0 ) numeroEnCirculo++;
    }
    return numeroEnCirculo;  
}
"""

# se crea el archivo con permisos para escribir mediante python
archivo_texto = open("aprox_PI.c", "w")
# se escribe el programa en el archivo 
archivo_texto.write(codigo)
# se cierra el buffer de escritura
archivo_texto.close()

In [ ]:
!mpicc aprox_PI.c -o aprox_PI

In [ ]:
!mpirun --allow-run-as-root -np 100 ./aprox_PI 100000

Numero de lanzamientos:      	    100000
Valor aproximado de pi:             3.1920000000000002
Maximo tiempo transcurrido del procesador en segundos:  0.0131242000009024


## Análisis a secciones importantes

*MPI_Abort(MPI_COMM_WORLD,1)*: en caso de que el usuario no introdusca los valores correctos el programa finaliza y todos los procesos asociados al mismo.

*MPI_Barrier(MPI_COMM_WORLD)*: esta directiva obliga a que todos los hilos asignados a este procesador terminen antes de poder avanzar.

<img src="https://github.com/jugernaut/ProgramacionEnParalelo/blob/main/Imagenes/MPI/barrier.png?raw=1" width="700">

*MPI_Allreduce(&dardo_dentro_proc, &total_dardos_dentro, 1, MPI_INT, MPI_SUM, MPI_COMM_WORLD)*: funciona de manera simular al reduce de OpenMP en el cual se realiza la reduccion de multiples variables en una sola.

# Suma de los primeros $n$ números naturales

En esta sección vamos a analizar un algoritmo ampliamente conocido y hasta cierto punto sencillo, sin embargo la idea de este seminario es mostrar las cualidades de la programación en paralelo y la suma de los primeros $n$ números naturales es un ejemplo ideal para ejemplificar el concepto de pase de mensajes entre procesos.

A primera vista esta labor puede parecer algo trivial.

$$\sum_{i=1}^{n}i=\frac{n(n+1)}{2}$$

Sin embargo la complejidad de esta sección radica en, ¿cómo dividir esta labor en diferentes secciones en paralelo para poder reducir el tiempo de ejecución. La idea sería compartir entre todos los procesos (broadcast) el vector de elementos y realizar la suma parcial en cada proceso respectivo.

<img src="https://github.com/jugernaut/ProgramacionEnParalelo/blob/main/Imagenes/MPI/broadcast.png?raw=1" width="700">




In [9]:
codigo = """
#include <stdio.h>
#include <mpi.h>
#define tam 10000

int main (int argc, char** argv){
   // bloque de variables globales
   int id_procesador, num_procesadores, a[tam], i, primero, ultimo, tam_porcion;
   double suma_total = 0, suma_parcial = 0;
   MPI_Status status;
   MPI_Init(&argc, &argv);
   MPI_Comm_rank(MPI_COMM_WORLD, &id_procesador);
   MPI_Comm_size(MPI_COMM_WORLD, &num_procesadores);
   // el proceso root (maestro) llena el vector a sumar
   if (id_procesador == 0) 
      for (i=0;i<tam;i++)
         a[i] = i + 1;
   // se transmite el vector a todos los procesos
   MPI_Bcast(a,tam,MPI_INT,0,MPI_COMM_WORLD);
   // determinamos la seccion a sumar dependiendo del proceso
   tam_porcion = tam/num_procesadores;
   // en caso de ser el ultimo procesador se asignan los limites
   if (id_procesador == num_procesadores - 1) {
      primero = (num_procesadores - 1)*tam_porcion;
      ultimo = tam - 1;
   } // en otro caso
   else {
      primero = id_procesador*tam_porcion;
      ultimo = (id_procesador + 1)*tam_porcion - 1;
   }
   // se realiza la suma parcial
   for (i=primero; i<=ultimo; i++) 
      suma_parcial = suma_parcial + a[i];
   // se juntan todas las sumas parciales
   MPI_Reduce(&suma_parcial, &suma_total, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD);
   if (id_procesador == 0) 
      printf("La suma es: %10.0f\\n", suma_total);
   // se liberan los recursos utilizados
   MPI_Finalize();
}

"""
# se crea el archivo con permisos para escribir mediante python
archivo_texto = open("sum_n.c", "w")
# se escribe el programa en el archivo 
archivo_texto.write(codigo)
# se cierra el buffer de escritura
archivo_texto.close()

In [10]:
!mpicc sum_n.c -o sum_n

In [11]:
!mpirun --allow-run-as-root -np 100 ./sum_n

La suma es:   50005000


## Nuevas funciones (broadcast)

Una de las nuevas funciones es la función *MPI_Bcast(a,tam,MPI_INT,0,MPI_COMM_WORLD);*, está se encarga de difundir (broadcast) el mensaje que incluyen los datos a procesar.

Una vez que cada proceso tiene la información pertinente, solo es cuestión de procesarla (realizar la suma parcial) y finalmente devolver el resultado mediante *reduce*.

# Referencias

*   [Teorema Fundamental de Monte Carlo](https://www.ugr.es/~jillana/Docencia/FM/mc.pdf).
*   [Ejemplos MPI (mallas)](https://htor.inf.ethz.ch/teaching/mpi_tutorials/ppopp13/2013-02-24-ppopp-mpi-basic.pdf)
*   [Ejemplos MPI (juego de la vida)](https://cimec.org.ar/~mstorti/curso-mpi-petsc/slides.pdf)